In [1]:
import pathlib
from typing import List

import gym
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import pandas as pd
from lcs.agents.acs2 import Configuration, ACS2
from lcs.metrics import population_metrics
from lcs.strategies.action_selection import EpsilonGreedy, ActionDelay, KnowledgeArray

from src.decorators import repeat, get_from_cache_or_run
from src.utils import build_plots_dir_path, build_cache_dir_path
from src.visualization import biased_exploration_colors, PLOT_DPI

COLORS = biased_exploration_colors()

plt.ioff()  # turn off interactive plotting

root_dir = pathlib.Path().cwd().parent.parent.parent
cwd_dir = pathlib.Path().cwd()

plot_dir = build_plots_dir_path(root_dir) /  cwd_dir.name
cache_dir = build_cache_dir_path(root_dir) /  cwd_dir.name


def run_alternating_experiment(env_provider, trials, **conf):
    env = env_provider()
    env.reset()

    cfg = Configuration(**conf)

    agent = ACS2(cfg)
    metrics = agent.explore_exploit(env, trials)

    # parse metrics
    lst = [[d['trial'], d['reward'], d['population'], d['reliable']] for d in metrics]
    df = pd.DataFrame(lst, columns=['trial', 'reward', 'population', 'reliable'])

    # df = df.set_index('trial')
    df['phase'] = df.index.map(lambda t: "explore" if t % 2 == 0 else "exploit")

    return df


def average_experiment_runs(runs_dfs: List[pd.DataFrame]) -> pd.DataFrame:
    return pd.concat(runs_dfs).groupby(['trial', 'phase']).mean().reset_index(level='phase')


def plot_rmpx(epsilon_greedy_df, action_delay_df, knowledge_array_df, op_initial_df, env_name, bins):
    def plot_by_phase(df, window, label, color, ax):
        explore_df = df[df['phase'] == 'explore']
        exploit_df = df[df['phase'] == 'exploit']

        explore_df.reset_index(inplace=True)
        exploit_df.reset_index(inplace=True)

        explore_df['reward'].rolling(window=window).mean().plot(label=label, color=color, ls='--', alpha=0.2, ax=ax)
        exploit_df['reward'].rolling(window=window).mean().plot(label=label, color=color, ax=ax)

    fig = plt.figure(figsize=(14, 10))

    # Plots layout
    gs = fig.add_gridspec(2, 1, hspace=.8)
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])

    # Global title
    fig.suptitle(f'Performance of [{env_name}] environment discretized with {bins} bins', fontsize=24)

    # Each axis
    ma_window = 500  # moving average window

    # Average reward
    plot_by_phase(epsilon_greedy_df, ma_window, 'Epsilon Greedy', COLORS['eg'], ax1)
    plot_by_phase(action_delay_df, ma_window, 'Action Delay', COLORS['ad'], ax1)
    plot_by_phase(knowledge_array_df, ma_window, 'Knowledge Array', COLORS['ka'], ax1)
    plot_by_phase(op_initial_df, ma_window, 'Optimistic Initial Quality', COLORS['oiq'], ax1)

    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.set_title('Average Reward')
    ax1.set_xlabel('Trial')
    ax1.set_ylabel('Reward')
    ax1.set_ylim(300, 1050)
    ax1.axhline(y=1000, color='black', linewidth=1, linestyle="--")

    # Population
    epsilon_greedy_df['reliable'].rolling(window=ma_window).mean().plot(label='Epsilon Greedy', c=COLORS['eg'], ax=ax2)
    action_delay_df['reliable'].rolling(window=ma_window).mean().plot(label='Action Delay', c=COLORS['ad'], ax=ax2)
    knowledge_array_df['reliable'].rolling(window=ma_window).mean().plot(label='Knowledge Array', c=COLORS['ka'],
                                                                         ax=ax2)
    op_initial_df['reliable'].rolling(window=ma_window).mean().plot(label='Optimistic Initial Quality', c=COLORS['oiq'],
                                                                    ax=ax2)

    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.set_xlabel('Trial')
    ax2.set_ylabel('Classifiers count')
    ax2.set_title('Reliable Classifiers')
    ax2.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.0f'))
    ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0f'))

    # Create legend
    handles, labels = ax2.get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=4)

    # Save plot to file
    fig.savefig(f'{plot_dir}/rmpx-performance.png', dpi=PLOT_DPI)

# Experiment 2 - Single-step problem performance

## Setup

In [2]:
from src.observation_wrappers import BinnedObservationWrapper

rmpx_bits = 6  # available sizes: 3, 6

def rmpx_metrics(agent, env):
    pop = agent.population
    metrics = {
        'reliable': len([cl for cl in pop if cl.is_reliable()])
    }
    metrics.update(population_metrics(pop, env))
    return metrics


def rmpx_env_provider(bins):
    import gym_multiplexer  # noqa: F401
    return BinnedObservationWrapper(gym.make(f'real-multiplexer-{rmpx_bits}bit-v0'), bins)


rmpx_base_params = {
    "classifier_length": rmpx_bits + 1,
    "number_of_possible_actions": 2,
    "epsilon": 0.8,
    "beta": 0.2,
    "gamma": 0.95,
    "initial_q": 0.5,
    "theta_exp": 50,
    "theta_ga": 50,
    "do_ga": True,
    "chi": 0.0,
    "mu": 0.03,
    "u_max": 4,
    "metrics_trial_frequency": 1,
    "user_metrics_collector_fcn": rmpx_metrics
}

# Start experiments
NUM_EXPERIMENTS = 50
USE_RAY = True

rmpx_trials = 2 * 15_000  # explore-exploit-explore-...

bins_v1 = 5
bins_v2 = 6
bins_v3 = 7

def run_rmpx_biased_exploration(bins):
    env_provider = lambda: rmpx_env_provider(bins)

    eg = run_alternating_experiment(env_provider,
                                    rmpx_trials,
                                    **(rmpx_base_params | {'action_selector': EpsilonGreedy}))

    ad = run_alternating_experiment(env_provider,
                                    rmpx_trials,
                                    **(rmpx_base_params | {'action_selector': ActionDelay,
                                                           'biased_exploration_prob': 0.5}))

    ka = run_alternating_experiment(env_provider,
                                    rmpx_trials,
                                    **(rmpx_base_params | {'action_selector': KnowledgeArray,
                                                           'biased_exploration_prob': 0.5}))

    oiq = run_alternating_experiment(env_provider,
                                     rmpx_trials,
                                     **(rmpx_base_params | {'action_selector': EpsilonGreedy,
                                                            'biased_exploration_prob': 0.8}))

    return eg, ad, ka, oiq

@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx/bins_{bins_v1}.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def rmpx_bins_v1():
    return run_rmpx_biased_exploration(bins_v1)

@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx/bins_{bins_v2}.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def rmpx_bins_v2():
    return run_rmpx_biased_exploration(bins_v2)

@get_from_cache_or_run(cache_path=f'{cache_dir}/rmpx/bins_{bins_v3}.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def rmpx_bins_v3():
    return run_rmpx_biased_exploration(bins_v3)

## Results

In [3]:
def extract(experiment_runs):
    eg_dfs, ad_dfs, ka_dfs, oiq_dfs = [], [], [], []

    for eg_df, ad_df, ka_df, oiq_df in experiment_runs:
        eg_dfs.append(eg_df)
        ad_dfs.append(ad_df)
        ka_dfs.append(ka_df)
        oiq_dfs.append(oiq_df)

    return eg_dfs, ad_dfs, ka_dfs, oiq_dfs


# Run calculations
rmpx_v1_eg_dfs, rmpx_v1_ad_dfs, rmpx_v1_ka_dfs, rmpx_v1_oiq_dfs = extract(rmpx_bins_v1())
rmpx_v2_eg_dfs, rmpx_v2_ad_dfs, rmpx_v2_ka_dfs, rmpx_v2_oiq_dfs = extract(rmpx_bins_v2())
rmpx_v3_eg_dfs, rmpx_v3_ad_dfs, rmpx_v3_ka_dfs, rmpx_v3_oiq_dfs = extract(rmpx_bins_v3())

# Plot results
plot_rmpx(
    average_experiment_runs(rmpx_v1_eg_dfs),
    average_experiment_runs(rmpx_v1_ad_dfs),
    average_experiment_runs(rmpx_v1_ka_dfs),
    average_experiment_runs(rmpx_v1_oiq_dfs),
    env_name=rmpx_env_provider(bins_v1).unwrapped.spec.id,
    bins=bins_v1
)

Here will be tabbed performance plot

## Statistical verification

```{admonition} Hypothesis testing
:class: tip
Novel idea would be to introduce a metric combining both average reward and number of classifiers (like _efficiency index_). Then such indicator might be used to plot the change when modifying number of bins. Otherwise a regular table might be used.
```

## Observations
...

---

**Software packages used**

In [ ]:
import session_info

session_info.show()